# Libraries

In [35]:
import sqlite3
import requests
import pandas as pd
from M_db import save_jobs_to_db
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import random
import time
from urllib.parse import quote
import pytz

In [37]:
# https://mostaql.com/projects?keyword=علوم%20البيانات&budget_max=1000&sort=latest

### Create a New Requests Session

In [40]:
session = requests.Session()

### Set a New SOCKS5 Proxy with Tor
This function sets a new SOCKS5 proxy with random credentials

In [43]:
def set_new_proxy():
    port = 9150 
    credentials = f"{random.randint(10000, 99999)}:pass"
    proxy_url = f"socks5h://{credentials}@127.0.0.1:{port}"
    session.proxies = {'http': proxy_url,
                       'https': proxy_url}

In [45]:
def check_tor_ip():
        response = session.get("http://httpbin.org/ip", timeout=10)
        print("Tor IP:", response.text)
check_tor_ip()

Tor IP: {
  "origin": "156.194.38.225"
}



### Generate Search URL for Mostaql
Builds a Mostaql search URL using a given keyword

In [48]:
def generate_url(keyword):
    encoded_keyword = quote(keyword)
    url = f"https://mostaql.com/projects?keyword={encoded_keyword}&budget_max=1000&sort=latest"
    return url

### Fetch Job Data with Tor & Retry Logic
Sends a GET request to the provided URL using a Tor proxy

In [51]:
def get_job_data(url, max_retries=5):
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'X-Requested-With': 'XMLHttpRequest'}
    for attempt in range(max_retries):
        try:
            set_new_proxy()
            check_tor_ip()
            response = session.get(url, headers=headers, timeout=15)
            data = response.json()
            time.sleep(random.uniform(3, 6))
            return data['collection']
        except Exception as e:
            print(f"محاولة {attempt+1} فشلت: {e}")
            time.sleep(5)
    return []

### Parse Job Project Details from HTML
Extracts key details from a Mostaql project using BeautifulSoup

In [54]:
def parse_job_details(project):
    html = project['rendered']
    soup = BeautifulSoup(html, 'html.parser')

    project_id = project['id']

    title_tag = soup.find('a')
    title = title_tag.text.strip() if title_tag else 'غير متوفر'
    
    description_tag = soup.find('p')
    description = description_tag.text.strip() if description_tag else 'غير متوفر'
    
    date_tag = soup.find('time')
    if date_tag and 'datetime' in date_tag.attrs:
        utc_time_str = date_tag['datetime']
        utc_time = datetime.strptime(utc_time_str, '%Y-%m-%d %H:%M:%S')
        utc_time = utc_time.replace(tzinfo=pytz.utc)
        local_time = utc_time.astimezone(pytz.timezone('Africa/Cairo'))
        date = local_time.strftime('%Y-%m-%d %H:%M:%S')
    else:
        date = 'غير متوفر'

    link = title_tag['href'] if title_tag else '#'
    
    return {
        'id': project_id,
        'title': title,
        'description': description,
        'date': date,
        'link': link
    }

### Extract Budget and Skills from Project Page

In [57]:
def extract_additional_details(project_url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        response = session.get(project_url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.content, 'html.parser')

        budget_tag = soup.find('span', dir='rtl')
        budget = budget_tag.text.strip() if budget_tag else 'غير متوفر'

        skills_list = []
        skills_section = soup.find('ul', class_='skills')
        if skills_section:
            for li in skills_section.find_all('li'):
                skill = li.get_text(strip=True)
                skills_list.append(skill)

        return {
            'budget': budget,
            'skills': ', '.join(skills_list)
        }

    except Exception as e:
        print(f"فشل في جلب التفاصيل من {project_url}: {e}")
        return {
            'budget': 'غير متوفر',
            'skills': 'غير متوفر'
        }

### Scrape Mostaql Jobs by Keyword

In [60]:
def scrape_jobs(keyword):
    url = generate_url(keyword)
    projects = get_job_data(url)

    if projects:
        job_list = []
        for project in projects:
            job_details = parse_job_details(project)
            extra_details = extract_additional_details(job_details['link'])
            job_details.update(extra_details)
            job_list.append(job_details)
        
        df = pd.DataFrame(job_list)
        print(df.head())
        save_jobs_to_db(df)
        return df

## هتدخل اسم الوظيفة بالعربي هنا

In [63]:
scrape_jobs("علوم البيانات")

Tor IP: {
  "origin": "185.100.87.192"
}

        id                                              title  \
0  1081222                رفع إقرار زكوي - شركة لم تعمل صفرية   
1  1081144                       استخراج بيانات من Google Map   
2  1081145                              أخصائي إدخال البيانات   
3  1081130                      تطبيق أندرويد للتحكم بالراوتر   
4  1077714  محترف لادارة والإشراف على محتوى منصات التواصل ...   

                                         description                 date  \
0  السلام عليكم عندي شركة (لم تعمل) ولا يوجد اي ح...  2025-04-27 22:45:31   
1  مرحبا اريد عمل استخراج بيانات لمشرويع موجوة في...  2025-04-27 19:57:14   
2  نحتاج إلى أخصائي إدخال بيانات ماهر لإدخال المع...  2025-04-27 19:47:06   
3  تطبيق أندرويد للتحكم بالراوترات يدعم تسجيل الد...  2025-04-27 19:22:19   
4  إدارة فريق التواصل الاجتماعي وتطوير الأداء الر...  2025-04-27 19:19:19   

                                                link               budget  \
0  https://mostaql.com/proj

,id,title,description,date,link,budget,skills
0,1081222,رفع إقرار زكوي - شركة لم تعمل صفرية,السلام عليكم عندي شركة (لم تعمل) ولا يوجد اي ح...,2025-04-27 22:45:31,https://mostaql.com/project/1081222-%D8%B1%D9%...,$50.00 - $100.00,"استشارات مالية, ضبط الحسابات, التحليل المالي, ..."
1,1081144,استخراج بيانات من Google Map,مرحبا اريد عمل استخراج بيانات لمشرويع موجوة في...,2025-04-27 19:57:14,https://mostaql.com/project/1081144-%D8%A7%D8%...,$50.00 - $100.00,"بايثون, web scraping, استخراج البيانات, معالجة..."
2,1081145,أخصائي إدخال البيانات,نحتاج إلى أخصائي إدخال بيانات ماهر لإدخال المع...,2025-04-27 19:47:06,https://mostaql.com/project/1081145-%D8%A3%D8%...,$1000.00 - $2500.00,"هندسة البرمجيات, إدخال بيانات, تحليل البيانات,..."
3,1081130,تطبيق أندرويد للتحكم بالراوتر,تطبيق أندرويد للتحكم بالراوترات يدعم تسجيل الد...,2025-04-27 19:22:19,https://mostaql.com/project/1081130-%D8%AA%D8%...,$50.00 - $100.00,"adobe xd, تصميم تطبيقات الهواتف الذكية, تصميم ..."
4,1077714,محترف لادارة والإشراف على محتوى منصات التواصل ...,إدارة فريق التواصل الاجتماعي وتطوير الأداء الر...,2025-04-27 19:19:19,https://mostaql.com/project/1077714-%D9%85%D8%...,$100.00 - $250.00,"التسويق عبر مواقع التواصل الاجتماعي, التسويق ا..."
5,1081119,تطبيق designer للتصاميم,تصاميمي بلس (أو بالإنجليزي: MyDesigns) --- الو...,2025-04-27 18:43:36,https://mostaql.com/project/1081119-%D8%AA%D8%...,$500.00 - $1000.00,"هندسة البرمجيات, تصميم تجربة المستخدم, تصميم و..."
6,1080849,منصة للقوائم الإلكترونية (المنيو الإلكتروني),السلام عليكم، نخطط لتسجيل شركة جديدة في السعود...,2025-04-27 18:41:09,https://mostaql.com/project/1080849-%D9%85%D9%...,$100.00 - $250.00,"هندسة البرمجيات, تطوير البرمجيات, برمجة مواقع,..."
7,1081097,تصميم لوحات تحكم لموقع إدارة عقارات وطلبات صيا...,مرحبا، لدي قالب مدفوع وأرغب في تعديله وتصميم م...,2025-04-27 18:05:08,https://mostaql.com/project/1081097-%D8%AA%D8%...,$50.00 - $100.00,"تصميم واجهة المستخدم, تصميم UI UX, Figma, التص..."
8,1081095,حملة تسويقية لموقع إلكتروني,احتاج البدء في إطلاق حملة تسويقية تشمل التعريف...,2025-04-27 17:52:05,https://mostaql.com/project/1081095-%D8%AD%D9%...,$250.00 - $500.00,"خطة تسويقية, التسويق الإلكتروني, التسويق عبر ا..."
9,1081085,إدارة حسابات السوشل ميديا لمنتجات لقيمات أو ز...,في حاجة شخص موهوب ومتمرس لإدارة حساباتنا على ا...,2025-04-27 17:37:02,https://mostaql.com/project/1081085-%D8%A5%D8%...,$25.00 - $50.00,"التسويق عبر مواقع التواصل الاجتماعي, إدارة حسا..."


# start to Summarize the description model

In [66]:
#pip install transformers
#pip install torch

In [82]:
import sqlite3

conn = sqlite3.connect("MostaqelJobs.db")
query = "SELECT description FROM jobs"

db_data = pd.read_sql_query(query, conn)




In [84]:
db_data

,description
0,السلام عليكم عندي شركة (لم تعمل) ولا يوجد اي ح...
1,مرحبا اريد عمل استخراج بيانات لمشرويع موجوة في...
2,نحتاج إلى أخصائي إدخال بيانات ماهر لإدخال المع...
3,تطبيق أندرويد للتحكم بالراوترات يدعم تسجيل الد...
4,إدارة فريق التواصل الاجتماعي وتطوير الأداء الر...
5,تصاميمي بلس (أو بالإنجليزي: MyDesigns) --- الو...
6,السلام عليكم، نخطط لتسجيل شركة جديدة في السعود...
7,مرحبا، لدي قالب مدفوع وأرغب في تعديله وتصميم م...
8,احتاج البدء في إطلاق حملة تسويقية تشمل التعريف...
9,في حاجة شخص موهوب ومتمرس لإدارة حساباتنا على ا...


In [86]:
# i want to know what is the length of the 29 lines 
for line in range( len(db_data)) :
    print(f' line number {line}   has length : { len(db_data['description'][line]) } ')

 line number 0   has length : 312 
 line number 1   has length : 212 
 line number 2   has length : 291 
 line number 3   has length : 114 
 line number 4   has length : 453 
 line number 5   has length : 453 
 line number 6   has length : 453 
 line number 7   has length : 453 
 line number 8   has length : 387 
 line number 9   has length : 242 
 line number 10   has length : 453 
 line number 11   has length : 453 
 line number 12   has length : 346 
 line number 13   has length : 328 
 line number 14   has length : 227 
 line number 15   has length : 453 
 line number 16   has length : 453 
 line number 17   has length : 453 
 line number 18   has length : 260 
 line number 19   has length : 453 
 line number 20   has length : 453 
 line number 21   has length : 453 
 line number 22   has length : 453 
 line number 23   has length : 453 
 line number 24   has length : 453 


In [88]:
from transformers import pipeline
summarizer = pipeline('summarization', model="google/pegasus-xsum", framework="pt")

summaries = []


for line in db_data["description"].tolist():
    i=0
    if len(line) > 512:
        line = line[:512]  

    
    try:
        summary = summarizer(line, max_length=60, min_length=10, do_sample=False)
        summaries.append(summary[0]['summary_text'])
        print(f' t {i}')
        print(summaries)
        i=i+1
    except Exception as e:
        summaries.append("")  
        print(f' e {i}')
        print(summaries)
        i=i+1

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


 t 0
['                            ']
 t 0
['                            ', '        800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217']


Your max_length is set to 60, but your input_length is only 34. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)


 t 0
['                            ', '        800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217', '          .']
 t 0
['                            ', '        800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-2

In [89]:
len(summaries)

25

In [94]:
# ad to DB

con=sqlite3.connect('MostaqelJobs.db')
cursor=con.cursor()
cursor.execute('ALTER TABLE jobs ADD COLUMN summary TEXT')
conn.commit()

In [96]:
conn = sqlite3.connect('MostaqelJobs.db')
cursor = conn.cursor()

for i, summary in enumerate(summaries):
    cursor.execute('''
        UPDATE jobs
        SET summary = ?
        WHERE rowid = ?''', (summary, i + 1))  


conn.commit()


In [100]:
conn = sqlite3.connect('MostaqelJobs.db')
cursor = conn.cursor()


cursor.execute("SELECT rowid, description, summary FROM jobs LIMIT 5")
rows = cursor.fetchall()


for row in rows:
    print(f'Row ID: {row[0]} \n , Description: {row[1]} , \n Summary: {row[2]} \n ')

conn.close()

Row ID: 1 
 , Description: السلام عليكم عندي شركة (لم تعمل) ولا يوجد اي حركة مالية - وطوفت ثلاث سنوات وابغى ارفع الإقرارات الزكوية تبعها - صفرية ولم تعمل سابقا وودي ترفعوا الإقرارات عشان ابي اشطبها مطلوب شخص يعمل في شركة معتمدة محاسبيا في المملكة العربية السعودية - مع وجود خبره سابقة في الإقرارات الزكوية للشركات التي لم تعمل وجاهزة للشطب , 
 Summary:                              
 
Row ID: 2 
 , Description: مرحبا اريد عمل استخراج بيانات لمشرويع موجوة في جوجل ماب وتشمل البيانات كل المعلومات عن الانشطة المستهدف بالصور والمعلومات لو عندك خبرة ممتازة وتشرح لي تجاربك السابقة يكون ممتاز البيانات تقريبا 20 الف نشاط ( شركة) , 
 Summary:         800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800-273-3217 800